# Apache Spark for astronomy: hands-on session 2

### Context

Welcome to the series of notebooks on Apache Spark! The main goal of this series is to get familiar with Apache Spark, and in particular its Python API called PySpark in a context of the astronomy. In this second notebook, we will learn on concrete examples how to interface and play with popular scientific libraries (Numpy, Pandas, ...).

### Learning objectives

- Interfacing popular Python scientific libraries with Apache Spark
- Developing your own modules for Spark

Through this series of exercises, we will use the same dataset as in the first session:

In [1]:
# Load data into a Spark DataFrame
df = spark.read.format("parquet").load("../data/clusters.parquet")

In [12]:
df.columns[:-1]

['x', 'y', 'z']

## User defined functions and column creation

Similarly to `map` and `mapPartitions`, you would like to define your own functions but this time to create new DataFrame columns. In python, the efficient way of doing this is via "Pandas User Defined Functions" (vectorized functions). 

**Exercise (£):** Use pandas UDF to compute the distance of each row to the center (x, y, z) = (0, 0, 0), and store the result in a new Dataframe column:

In [28]:
from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.sql.types import DoubleType
import numpy as np
import pandas as pd

In [ ]:
@pandas_udf('double', PandasUDFType.SCALAR)
def distance_mod(cols):
    
    x = cols.select('x')
    y = cols.select('y')
    z = cols.select('z')
    
    r_sq = x*x + y*y + z*z
    return pd.Series(np.sqrt(r_sq))
    

df.withColumn("distance", distance_mod(df.select('x', 'y', 'z')))

In [40]:
@pandas_udf('double', PandasUDFType.SCALAR)
def distance_mod(x, y, z):
    
    r_sq = x*x + y*y + z*z
    return pd.Series(np.sqrt(r_sq))
    
    
df.withColumn(
    "distance", 
    distance_mod(
        col("x"), 
        col("y"), 
        col("z")
    )
).show(5)

+-------------------+-------------------+------------------+---+------------------+
|                  x|                  y|                 z| id|          distance|
+-------------------+-------------------+------------------+---+------------------+
|-1.4076402686194887|  6.673344773733206| 8.208460943517498|  2|10.672104415540714|
| 0.6498424376672443|  3.744291410605022|1.0917784706793445|  0|3.9539845207540685|
| 1.3036201950328201|-2.0809475280266656| 4.704460741202294|  1|5.3067616389669645|
|-1.3741641126376476|  4.791424573067701| 2.562770404033503|  0| 5.604807631993114|
| 0.3454761504864363| -2.481008091382492|2.3088066072973583|  1|3.4066615432062313|
+-------------------+-------------------+------------------+---+------------------+
only showing top 5 rows



**Exercise (£££):** As in session 1, find the barycentre of each clusters in the dataset but this time using aggregation and user defined function (hint: look for `PandasUDFType.GROUPED_MAP`). 

In [45]:
@pandas_udf(df.schema, PandasUDFType.GROUPED_MAP)
def barycentre(dataframe: pd.DataFrameta):
    
    mean = dataframe.mean()
    # This is just to reconstruct a Pandas DataFrames with the result
    # i: column name, j: mean value [list of x, y, z]
    out = {i : [j] for i, j in zip(mean.keys(), mean.values)}
    return pd.DataFrame(out)
    
df.groupBy("id").apply(barycentre).show()

+-------------------+-------------------+------------------+---+
|                  x|                  y|                 z| id|
+-------------------+-------------------+------------------+---+
| 0.9084311322436593|-1.5335608883132903| 2.926201255363395|  1|
|-1.2364938227997018| 7.7837163227456205| 9.292937669035544|  2|
|  1.001314312562809|  4.250879907797302|2.0216900721305446|  0|
+-------------------+-------------------+------------------+---+

